<a href="https://colab.research.google.com/github/mancap314/miscellanous/blob/master/cnn_estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating a tf `Estimator` from a Keras model

In [0]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np



## Step 1: Importing data

In [0]:
# 1.0: Define some constants first
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)
num_classes = 10

# 1.1: Import data
((train_data, train_labels), (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

# 1.2: Format data
# convert class vectors to binary class matrices
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
eval_labels = tf.keras.utils.to_categorical(eval_labels, num_classes)

train_data = train_data / np.float32(255.0)
eval_data = eval_data /np.float32(255.0)

train_data = train_data.reshape(train_data.shape[0], img_rows, img_cols, 1)
eval_data = eval_data.reshape(eval_data.shape[0], img_rows, img_cols, 1)


##  Step 2: Build model

In [40]:
# 2.1: Define model
model_cnn_0 = tf.keras.models.Sequential()
model_cnn_0.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape,
                 name='x'
                ))
model_cnn_0.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model_cnn_0.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model_cnn_0.add(tf.keras.layers.Dropout(0.25))
model_cnn_0.add(tf.keras.layers.Flatten())
model_cnn_0.add(tf.keras.layers.Dense(128, activation='relu'))
model_cnn_0.add(tf.keras.layers.Dropout(0.5))
model_cnn_0.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model_cnn_0.summary()

# 2.2: Compile model
model_cnn_0.compile(loss=tf.keras.losses.categorical_crossentropy,
                    optimizer=tf.keras.optimizers.Adadelta(),
                    metrics=['accuracy'])


# 2.3: Transform the keras `model` into a tf `Estimator`
est_cnn_0 = tf.keras.estimator.model_to_estimator(keras_model=model_cnn_0)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
x (Conv2D)                   (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 9216)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               1179776   
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
__________

## Step 3: Train Model

In [44]:
# 3.1: Define training input function
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x_input': train_data},  # 'x_input' because name of 1st layer is 'x', `model_cnn_0.input_names`
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

# 3.2 Train estimator
est_cnn_0.train(input_fn=train_input_fn, steps=2000)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/tmp/tmpvm16sdmt/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ('/tmp/tmpvm16sdmt/keras/keras_model.ckpt',)
INFO:tensorflow:Warm-starting variable: x/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: x/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_19/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: conv2d_19/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_13/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_13/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_14/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dense_14/bias; prev_var_name:

## Evaluate (trained) model

In [45]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x_input': eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = est_cnn_0.evaluate(input_fn=eval_input_fn)
print(eval_results)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Starting evaluation at 2019-03-27T23:13:38Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /tmp/tmpvm16sdmt/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-27-23:13:39
INFO:tensorflow:Saving dict for global step 2000: categorical_accuracy = 0.9878, global_step = 2000, loss = 0.03232793
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: /tmp/tmpvm16sdmt/model.ckpt-2000
{'categorical_accuracy': 0.9878, 'loss': 0.03232793, 'global_step': 2000}
